# Progetto CS

Paolo Merola & Mattia Fratello

In [2]:
# Starting from the The Movie DataBase (TMDB) sample datasets:

import pandas as pd
import numpy as np

cr = pd.read_csv("C:/Users/pablo/Desktop/Computer Science (Python)/Progetto/tmdb_5000_credits.csv")
mv = pd.read_csv("C:/Users/pablo/Desktop/Computer Science (Python)/Progetto/tmdb_5000_movies.csv")
mv.rename(columns = {'id': 'movie_id'},inplace = True)
#pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
def check_uniqueness(df, column):
    return len(df[column]) == len(df[column].drop_duplicates())
check_uniqueness(cr,'movie_id') and check_uniqueness(mv,'movie_id')

True

In [3]:
#FUNZIONE PER ESTRARRE I DATAFRAME SCRITTI IN JSON
def extract_json(df, column):
    result = df[column].apply(pd.read_json) #result è una Series di Dataframe
    movie_id = df['movie_id'] #Series degli ID dei film, nello stesso ordine dei dataframe contenuti in result
    for i in range(len(df)):
        result[i]['movie_id'] = movie_id[i] #attacco ad ogni dataframe l'id del film a cui si riferisce
    return pd.concat(result.values).reset_index().drop("index", axis = 1) #dalla Series di Dataframe ricavo un unico Dataframe

In [4]:
#ESTRAGGO TUTTI I JSON ED ELIMINO LE CORRISPONDENTI COLONNE NEI DATAFRAME INIZIALI
cast = extract_json(cr,'cast')

crew = extract_json(cr,'crew')

genres = extract_json(mv,'genres')

keywords = extract_json(mv,'keywords')

production_companies = extract_json(mv,'production_companies')

production_countries = extract_json(mv,'production_countries')

spoken_languages = extract_json(mv,'spoken_languages')

movies = mv.drop(['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages'],axis = 1)
credits = cr.drop(['cast', 'crew'], axis = 1)

RISOLUZIONE DELLE QUERY SULLE TABELLE NON NORMALIZZATE

In [9]:
#Rendo più leggibili cast e crew: rinomino le colonne con nomi parlanti, setto indici appropriati, 
# trasformo il gender da valore numerico a stringa e rimetto ad interi i valori castati a float per errore
cast.rename(columns = {'id': 'actor_id'}, inplace = True)
crew.rename(columns = {'id': 'crew_id'}, inplace = True)

cast.set_index('credit_id', inplace=True)
crew.set_index('credit_id',inplace=True)

cast.replace({"gender": {0.0: np.nan, 1.0: "F", 2.0: "M"}}, inplace = True)
crew.replace({"gender": {0.0: np.nan, 1.0: "F", 2.0: "M"}}, inplace = True)

cast[['cast_id', 'actor_id', 'order']] = cast[['cast_id','actor_id', 'order']].applymap(int)
crew['crew_id'] = crew['crew_id'].apply(int)

KeyError: 'credit_id'

In [6]:
cast.isnull().values.any()

True

In [10]:
# 1. For each movie, compute the number of cast members

actors_each_movie = cast.groupby(by = ['movie_id'], as_index = True).size().to_frame().rename(
    columns = {0:'number of actors'})
#actors_each_movie

In [11]:
# 2. How many movies do not have a homepage?
num_movies_no_homepage = len(mv[mv['homepage'].isnull()])
print(num_movies_no_homepage,'movies do not have a homepage')

3091 movies do not have a homepage


In [12]:
#3. For each year, how many movies do not have a homepage?
def extract_year(timestamp):
    if timestamp is np.nan:
        return np.nan
    else:
        return pd.to_datetime(timestamp).year

mv2 = mv[['homepage','release_date']].dropna(subset = ['release_date'])
mv2['release_year'] = mv2['release_date'].apply(extract_year) 
movies_per_year = mv2[mv2['homepage'].isnull()].groupby(by='release_year').size().to_frame().rename(columns = {0 : 'number_of_movies'})

In [13]:
#4. Extract the domain of each homepage (for example .com, .it, .org)

def extract_domain(hp):
    domains = []
    if hp is np.nan:
        return np.nan
    else:
        homepage = hp.split(' ')
        for i in range(len(homepage)):
            domains.append(homepage[i].split('/')[2].split('.')[-1])
        return str(domains)[1:-1].replace("\'", "")
    
domains = mv['homepage'].apply(extract_domain).to_frame()
domains_with_titles = mv['movie_id'].to_frame().join(domains).rename(columns = {'homepage':'domain'}).set_index('movie_id')

In [14]:
#6. For each movie, compute the gross margin (difference between revenue and budget)
mv['gross_margin'] =  mv['revenue'] - mv['budget']
gross_margin_movie = mv[['movie_id','gross_margin']].set_index('movie_id')
#gross_margin_movie

In [15]:
#7. For each movie, compute the number of crew members
num_crew_each_movie = crew.groupby(by = 'movie_id').size().to_frame().rename(columns = {0: 'num_crew_members'})
#num_crew_each_movie

In [16]:
#8. For each movie, compute the number of directors
num_crew_directors_movie = crew[crew['job']=='Director'].groupby(by = 'movie_id',as_index = True).size().to_frame().rename(
columns = {0: 'num_directors'})
#num_crew_directors_movie

In [17]:
#9. For each language, compute the number of movies where such language is spoken.
num_movies_each_language = spoken_languages.groupby(by = 'iso_639_1',as_index=True).size().to_frame().rename(columns = {0:'Number of movies where such language is spoken'})
#num_movies_each_language

In [18]:
#rendo più leggibile production_companies

production_companies.rename(columns = {'id': 'company_id'}, inplace = True)
production_companies['company_id'] = production_companies['company_id'].apply(int)

In [19]:
#10. For each company and each decade, compute the overall revenue
def extract_decade(timestamp):
    return str(extract_year(timestamp))[:3] + '0'

mv['release_decade'] = mv['release_date'].apply(extract_decade)

mv_companies = production_companies.merge(right = mv[['movie_id', 'revenue','release_decade']], on = 'movie_id')
revenue_company_decade = mv_companies[['company_id','revenue','release_decade']].groupby(by = ['company_id', 'release_decade']).sum()

In [20]:
# 11. For each decade, compute the company with maximum revenue

def extract_first_from_tuple(tup):
    return tup[0]
max_revenue_per_company_per_decade = revenue_company_decade.groupby(level=1).idxmax().rename(columns = {'revenue':'most_solvable_company'})
max_revenue_per_company_per_decade['most_solvable_company'] = max_revenue_per_company_per_decade['most_solvable_company'].apply(extract_first_from_tuple)

In [21]:
#12. In each year, how many movies have revenue smaller than the budget?

mv3 = mv[['release_date','gross_margin']].dropna(subset = ['release_date'])
mv3['release_year'] = mv3['release_date'].apply(extract_year) 
num_failure_movies_year = mv3[mv3['gross_margin'] < 0].groupby(by = 'release_year',as_index=True).size()

In [192]:
movies_revenue = mv_companies[['movie_id','revenue']].drop_duplicates()
cast2=pd.DataFrame()
cast2 = cast.merge(right = actors_each_movie.reset_index(), on = 'movie_id').merge(right = movies_revenue, on = 'movie_id')
cast2 = cast2[['actor_id','movie_id','order','number of actors','revenue']].rename(columns = {'number of actors':'num_actors'})
cast2.head()

,actor_id,movie_id,order,num_actors,revenue
0,65731,19995,0,83,2787965087
1,8691,19995,1,83,2787965087
2,10205,19995,2,83,2787965087
3,32747,19995,3,83,2787965087
4,17647,19995,4,83,2787965087


In [200]:
#13. Distribute the revenue according to the order of appearance in a movie. 
#    Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

#legge di distribuzione: 2^(n-k-1)/(2^n -1) * revenue 
#                        = [1/2^k * (1 + 1/(2^n -1))] * revenue
#                         = revenue/2^k + revenue/(2^k * (2^n -1))

def assign_part(orders,num_actors,revenues):
    #la funzione utilizza direttamente le Series con i valori necessari
    part = [0]*len(orders)
    for i in range(len(orders)):
        k = orders[i]
        n = num_actors[i]
        rev = revenues[i]
        e_n = np.exp2(n)
        e_k = np.exp2(k)
        
        number = rev/e_k + rev/(e_k * (e_n -1))
        part[i] = np.round(number/2)
        
    return part
cast2['part'] = assign_part(cast2['order'],cast2['num_actors'],cast2['revenue'])

In [201]:
#check: è vero che la somma delle parti dei vari attori corrisponde alla revenue totale?
movies_id = cast2['movie_id'].unique()
for movie_id in movies_id:
    print(cast2[cast2['movie_id'] == movie_id]['revenue'].iloc[0], cast2[cast2['movie_id'] == movie_id]['part'].sum())
    
#evidentemente si

2787965087 2787965087.0
961000000 961000000.0
880674609 880674608.0
1084939099 1084939099.0
284139100 284139100.0
890871626 890871625.0
591794936 591794936.0
1405403694 1405403694.0
933959197 933959197.0
873260194 873260193.0
391081192 391070003.0
586090727 586090168.0
1065659812 1065659812.0
89289910 89289910.0
662845518 662845518.0
419651413 419651413.0
1519557910 1519557910.0
1045713802 1045713802.0
624026776 624026776.0
956019788 956019788.0
752215857 752215856.0
310669540 310669540.0
958400000 958385376.0
372234864 372234864.0
550000000 550000000.0
1845034188 1845034188.0
1153304495 1153304495.0
303025485 303025449.0
1513528810 1513528810.0
1108561013 1108561013.0
783766341 783766340.0
1215439994 1215439993.0
1025491110 1025491110.0
459359555 459359555.0
743559607 743559607.0
836297228 836297228.0
1091405097 1091405097.0
491868548 491868547.0
705717432 705717432.0
400062763 400062763.0
559852396 559835294.0
219851172 219851172.0
1066969703 1066969703.0
371353001 371353001.0
150624

183018522 183018522.0
47664559 47664559.0
74208267 74208267.0
123307945 123307945.0
275650703 275650703.0
41037742 41037742.0
237681299 237681299.0
21028755 21028755.0
112483764 112483764.0
222231186 222231186.0
160388063 160388063.0
84785914 84785914.0
96085477 96085477.0
484572835 482797410.0
274470394 274453675.0
457696359 457695923.0
691210692 691210692.0
254455986 254455986.0
280000000 280000000.0
343397247 343397247.0
145771527 145771527.0
144000000 144000000.0
140464664 140464664.0
0 0.0
190320568 190320568.0
342695435 342695435.0
246984278 246019496.0
230685453 230685453.0
266728738 266728738.0
294804195 294804195.0
163733697 163733697.0
260696994 260693140.0
245676146 245676146.0
212874442 212874441.0
181001478 181001478.0
214918407 214918407.0
224012234 224012234.0
178066569 178066569.0
173013509 173013509.0
157107755 157107755.0
177977226 177977226.0
91188905 91188905.0
250200000 250077832.0
0 0.0
233093859 233093859.0
136982834 136982834.0
211817906 211817906.0
164000000 16

52304001 52304001.0
12923936 12923936.0
11466088 11466088.0
0 0.0
80383290 80383289.0
28283637 28283637.0
89519773 89519773.0
0 0.0
9250000 9250000.0
6416302 6416302.0
155010032 155010032.0
168167691 168167691.0
9489829 9488680.0
136766062 136766062.0
52034889 52034889.0
130431368 130431368.0
190458706 190458706.0
7266209 7266209.0
9450897 9450897.0
107196498 107196498.0
139804348 139804348.0
179213434 179213433.0
54249294 54249294.0
98837872 98837866.0
65900249 65900249.0
126690726 126690726.0
25037897 12518948.0
0 0.0
5761917 5761917.0
352114312 352114311.0
132820716 132820716.0
204999686 204999686.0
609841637 609841637.0
216485654 216485654.0
110803676 110803676.0
98126565 98126565.0
75450437 75155780.0
47434430 47434430.0
116112375 116105288.0
52826594 52826594.0
0 0.0
149270999 149270999.0
72779000 72770151.0
47231070 47231070.0
709827462 709827462.0
484409218 484409218.0
127869379 127869379.0
390493908 390464116.0
273552592 273552592.0
549368315 549368315.0
314178011 314178011.0


127814609 127814609.0
180622424 180622424.0
205298907 205298907.0
0 0.0
119723358 119723358.0
120000000 120000000.0
64076736 60934964.0
114156230 114156230.0
0 0.0
73956241 73956240.0
51801187 51801187.0
0 0.0
107597242 107597242.0
29700000 29700000.0
0 0.0
58401898 58401898.0
17380126 17380126.0
16991902 16983735.0
47536959 47536959.0
10300000 10300000.0
19629760 19629760.0
98337295 98337295.0
6482195 6457659.0
0 0.0
17869048 17868507.0
28258060 28258060.0
5410749 4058392.0
9793496 9793496.0
109906372 109906372.0
173567581 173567580.0
154026136 154026136.0
79700000 79700000.0
163720069 163720068.0
100076342 100076342.0
90247624 90247624.0
48451803 48451803.0
55583804 55583804.0
112343513 112178987.0
95404397 95404397.0
392616625 392616625.0
102244770 102196404.0
117758500 117758500.0
269994119 269962190.0
21834845 20299786.0
50159144 50159144.0
160000000 160000000.0
64572262 64572262.0
52424533 52219749.0
32168970 32168970.0
44350926 44350926.0
26250020 26250020.0
0 0.0
0 0.0
96047633

0 0.0
43984230 43973489.0
63100000 63100000.0
0 0.0
77208711 77208711.0
0 0.0
0 0.0
41604473 41604473.0
39053061 39053061.0
64437847 64437847.0
70442940 70442940.0
36351350 36351350.0
38307627 38307627.0
66980456 66980456.0
0 0.0
63102666 63102666.0
59192128 59192128.0
56489558 56489558.0
41597830 41595331.0
33889159 33889159.0
229736344 229736344.0
70992898 70992898.0
2415472 2414311.0
66122026 66122025.0
82150642 82150642.0
50671850 50671850.0
29548291 29432868.0
100000000 100000000.0
34942188 34942188.0
0 0.0
28341469 28341469.0
39438674 39429083.0
25023434 25023434.0
39171130 39171130.0
74188937 74188937.0
27645491 27645491.0
42886719 42886719.0
46152800 46152800.0
21567853 21567853.0
22200000 22200000.0
114977104 114977104.0
23580000 23580000.0
36993168 36993168.0
40452643 40452643.0
17000000 17000000.0
20796847 20796844.0
0 0.0
0 0.0
0 0.0
15431177 15431177.0
33333531 33329716.0
0 0.0
24362772 24362772.0
19829957 19829957.0
30411183 30411183.0
51702483 38574899.0
0 0.0
42972994 4

0 0.0
0 0.0
0 0.0
0 0.0
29536299 29521884.0
10000000 10000000.0
97542952 97542952.0
20455276 20455276.0
7314027 7314027.0
19300000 19300000.0
0 0.0
9500000 9500000.0
0 0.0
0 0.0
86915017 86915017.0
2000000 2000000.0
1569918 1569918.0
3758328 3758328.0
374743 374743.0
0 0.0
8720000 8720000.0
9627492 9627492.0
0 0.0
2281089 2281089.0
43848069 43848063.0
135000000 135000000.0
37311672 37311672.0
13620075 13620025.0
16942795 16942795.0
235860116 235860116.0
247275374 247275374.0
92823546 92823546.0
53977250 53977250.0
119754278 119754278.0
0 0.0
133432856 133432856.0
76514050 76514050.0
68263166 68263166.0
0 0.0
6490401 6490401.0
0 0.0
83258226 83258226.0
0 0.0
29381649 29381649.0
40492759 40492759.0
30987695 30987695.0
30102717 30102717.0
80573774 80573774.0
50007546 49922042.0
96408652 96408652.0
0 0.0
20627372 20627372.0
0 0.0
31556061 31556046.0
15427192 15427192.0
0 0.0
0 0.0
0 0.0
13854000 13854000.0
206678440 206678438.0
18564088 18564088.0
12 12.0
0 0.0
0 0.0
0 0.0
4217115 4216085.

38368909 38368909.0
25605015 25605015.0
33697647 33697647.0
143695338 143695338.0
20766616 20746272.0
52882018 52882018.0
24637469 24637469.0
0 0.0
42333295 42333295.0
0 0.0
0 0.0
0 0.0
0 0.0
17096053 17096053.0
66821036 66821036.0
85978266 85978266.0
0 0.0
0 0.0
16153593 16153593.0
0 0.0
48572000 48572000.0
28650575 28650575.0
15795189 15795189.0
15156200 15156200.0
21587700 21587700.0
20342161 19984584.0
0 0.0
0 0.0
0 0.0
35057696 35057696.0
17297244 17297244.0
0 0.0
58978653 58978653.0
0 0.0
0 0.0
12827813 12827813.0
10571408 10571408.0
28883511 28883511.0
9898412 9898412.0
8801940 8799825.0
6110000 6110000.0
0 0.0
32438988 32438988.0
0 0.0
9636289 9636289.0
6861102 6861102.0
23875127 23875127.0
6842058 6842058.0
0 0.0
9495408 9495408.0
0 0.0
0 0.0
7800000 7800000.0
33000000 33000000.0
2154696 2154696.0
21571189 21571189.0
0 0.0
7177143 6728574.0
10835752 10835752.0
4417580 4417580.0
0 0.0
0 0.0
3275585 3275585.0
3193102 3193102.0
98410061 98410049.0
5164404 5164404.0
0 0.0
0 0.0
0 

11744471 11744471.0
10000000 10000000.0
15179302 15179302.0
44349000 44348999.0
31584901 31584901.0
10229331 10229331.0
0 0.0
8200000 8200000.0
0 0.0
0 0.0
44000000 44000000.0
0 0.0
0 0.0
6000000 6000000.0
4630045 4630045.0
4300000 4300000.0
9304609 9304609.0
3468572 3468572.0
0 0.0
2426851 2426851.0
1919197 1919197.0
1104682 1104682.0
866999 866999.0
0 0.0
0 0.0
1406224 1406224.0
0 0.0
0 0.0
0 0.0
641100 641100.0
2365467 2365467.0
302204 302204.0
354704 354704.0
0 0.0
0 0.0
120000 120000.0
0 0.0
0 0.0
318982 318982.0
0 0.0
146402 146402.0
33245 33245.0
0 0.0
0 0.0
355688 344736.0
25345000 25345000.0
4037782 4037782.0
0 0.0
0 0.0
8000000 8000000.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
28350000 28349784.0
23689877 23689877.0
400176459 400176459.0
0 0.0
14873 14873.0
8000000 8000000.0
4270000 4270000.0
44462 44462.0
2956000 2933617.0
0 0.0
13401952 13401952.0
44785053 44785053.0
257850122 257849138.0
83453539 83453539.0
0 0.0
28215918 28215918.0
27900000 27900000.0
14000000 14000000.0
5000000 56

4358000 4358000.0
29400000 29400000.0
31081 31081.0
0 0.0
1750211 1750211.0
0 0.0
0 0.0
0 0.0
1000000 1000000.0
246574 246574.0
4500000 4500000.0
110197 110197.0
7808524 7808524.0
1281176 1281176.0
16 16.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
40542 40542.0
0 0.0
0 0.0
8000000 8000000.0
0 0.0
0 0.0
319285 319285.0
12021272 12021272.0
501818 501818.0
203 203.0
0 0.0
3542353 3542353.0
0 0.0
56825 56825.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
22000000 22000000.0
0 0.0
0 0.0
2260712 2258521.0
0 0.0
111300 111300.0
5000000 5000000.0
100000000 100000000.0
4505922 4505921.0
14500000 14500000.0
2200000 2200000.0
2295733 2295733.0
2611555 2611555.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
900000 900000.0
469947 469947.0
0 0.0
1776935 1776935.0
15200000 15200000.0
2891288 2891288.0
20710513 20710513.0
0 0.0
0 0.0
0 0.0
0 0.0
6706368 6706368.0
0 0.0
3200000 3200000.0
0 0.0
0 0.0
15861566 15861566.0
10178331 10178331.0
42664410 21332205.0
7000000 7000000.0
0 0.0
0 0.0
0 

In [203]:
#14. For each actor find the total revenue attributed to him/her.
cast2.groupby(by='actor_id').sum()['part']

actor_id
1          5.400000e+01
2          9.432493e+08
3          2.509048e+09
4          2.477412e+08
5          4.846238e+07
6          7.302262e+07
7          2.315620e+05
8          3.000000e+01
10         5.037045e+07
11         0.000000e+00
12         1.175430e+08
13         4.997938e+08
14         2.350839e+08
18         5.078511e+07
19         1.276659e+08
20         1.801125e+08
22         2.216200e+04
23         2.040500e+04
31         3.800124e+09
32         2.768167e+08
33         2.743127e+08
34         5.127522e+07
35         2.369315e+08
40         1.319868e+06
47         2.001594e+07
48         2.912840e+08
49         1.655558e+08
50         2.353529e+07
51         0.000000e+00
52         7.903366e+07
               ...     
1890145    0.000000e+00
1890146    0.000000e+00
1890147    0.000000e+00
1890148    0.000000e+00
1890149    0.000000e+00
1890150    0.000000e+00
1890151    0.000000e+00
1890374    0.000000e+00
1890375    0.000000e+00
1890376    0.000000e+00
1890377

In [204]:
#15. Find the actor that is responsible for the most overall revenue.
cast2.groupby(by='actor_id').sum()['part'].idxmax()

500

NORMALIZZAZIONE DELLE TABELLE DA QUI IN POI

In [54]:
#Normalizzo cast e crew

actors = cast[['actor_id' , 'name', 'gender']].set_index('actor_id')

workers = crew[['crew_id','name','gender']].set_index('crew_id')

cast.drop(['name', 'gender' ], axis = 1, inplace = True)

crew.drop(['name', 'gender' ], axis = 1, inplace = True)

In [65]:
genres.rename(columns = {'id': 'genre_id'}, inplace = True)
genres['genre_id'] = genres['genre_id'].apply(int)
genres_names = genres[['genre_id','name']].set_index('genre_id').drop_duplicates().sort_index()
genres.drop('name',inplace = True, axis = 1)
genres_names

In [72]:
keywords.rename(columns = {'id': 'keyword_id'}, inplace = True)
keywords['keyword_id'] = keywords['keyword_id'].apply(int)
keywords_names = keywords[['keyword_id','name']].set_index('keyword_id').drop_duplicates().sort_index()
keywords.drop('name',inplace = True, axis = 1)
keywords_names

,name
keyword_id,
30,individual
65,holiday
74,germany
75,gunslinger
83,saving the world
90,paris
100,slum
107,barcelona spain
108,transvestism


In [74]:
production_companies_names = production_companies[['company_id','name']].set_index('company_id').drop_duplicates().sort_index()
production_companies.drop('name',inplace = True, axis = 1)
production_companies_names

,name
company_id,
1,Lucasfilm
2,Walt Disney Pictures
3,Pixar Animation Studios
4,Paramount Pictures
5,Columbia Pictures
6,RKO Radio Pictures
7,DreamWorks
8,Fine Line Features
9,Gaumont


In [76]:
production_countries_names = production_countries[['iso_3166_1','name']].set_index('iso_3166_1').drop_duplicates().sort_index()
production_countries.drop('name',inplace = True, axis = 1)
production_countries_names

,name
iso_3166_1,
AE,United Arab Emirates
AF,Afghanistan
AO,Angola
AR,Argentina
AT,Austria
AU,Australia
AW,Aruba
BA,Bosnia and Herzegovina
BE,Belgium


In [83]:
spoken_languages_names = spoken_languages[['iso_639_1','name']].set_index('iso_639_1').drop_duplicates().sort_index()
spoken_languages.drop('name',inplace = True, axis = 1)
spoken_languages_names


,name
iso_639_1,
af,Afrikaans
ar,العربية
bg,български език
bm,Bamanankan
bn,বাংলা
bo,
bs,Bosanski
ca,Català
cn,广州话 / 廣州話
